In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy.sparse as sparse
from sklearn.preprocessing import MinMaxScaler

import implicit

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_parquet('./data/train_mfti.parquet', engine='pyarrow')
df

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
...,...,...,...,...,...,...
12292583,2022-08-11,1660246479,162851,0d3c58c6864546c689fa0997de484b30,18124998fd644e80b667a1d84bf63846,preview_click_response
12292584,2022-08-11,1660246106,136818,ef559274a81e4ff4b92aa3e9f6805886,5c71c1f1b7cf492ba05871c400a05250,preview_click_response
12292585,2022-08-11,1660250139,143721,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response
12292586,2022-08-11,1660250318,154423,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 6 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   event_date       object
 1   event_timestamp  int64 
 2   vacancy_id_      int64 
 3   cookie_id        object
 4   user_id          object
 5   event_type       object
dtypes: int64(2), object(4)
memory usage: 562.7+ MB


In [4]:
df['event_type'].unique()

array(['show_vacancy', 'preview_click_vacancy', 'click_contacts',
       'preview_click_contacts', 'click_favorite',
       'preview_click_favorite', 'preview_click_phone',
       'preview_click_response', 'click_phone', 'click_response'],
      dtype=object)

In [5]:
value = df['event_type'].value_counts()
value

event_type
show_vacancy              6198889
preview_click_vacancy     4781280
click_response             384090
click_contacts             277584
preview_click_response     190635
click_favorite             155844
preview_click_favorite     107016
preview_click_contacts     102050
click_phone                 79191
preview_click_phone         16009
Name: count, dtype: int64

In [6]:
k = []
for element in value:
    k.append(1 - (element / sum(value)))
k = pd.Series(k)
k.index = value.index
k

event_type
show_vacancy              0.495721
preview_click_vacancy     0.611044
click_response            0.968754
click_contacts            0.977419
preview_click_response    0.984492
click_favorite            0.987322
preview_click_favorite    0.991294
preview_click_contacts    0.991698
click_phone               0.993558
preview_click_phone       0.998698
dtype: float64

In [7]:
def rating(events, koef):
    data = []
    for event in events:
        data.append(k[event])
    return data

In [8]:
df['rating'] = rating(df.event_type, k)
df

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type,rating
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy,0.495721
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0.495721
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0.495721
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0.495721
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0.495721
...,...,...,...,...,...,...,...
12292583,2022-08-11,1660246479,162851,0d3c58c6864546c689fa0997de484b30,18124998fd644e80b667a1d84bf63846,preview_click_response,0.984492
12292584,2022-08-11,1660246106,136818,ef559274a81e4ff4b92aa3e9f6805886,5c71c1f1b7cf492ba05871c400a05250,preview_click_response,0.984492
12292585,2022-08-11,1660250139,143721,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response,0.984492
12292586,2022-08-11,1660250318,154423,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response,0.984492


In [9]:
df_new = df[['cookie_id', 'vacancy_id_', 'rating']]
df_new

,cookie_id,vacancy_id_,rating
0,97990f1a021d4be19aa3f955b7eacab4,129850,0.495721
1,03bf8c511fa949c79845a5d81b09aa1d,108347,0.495721
2,03bf8c511fa949c79845a5d81b09aa1d,109069,0.495721
3,03bf8c511fa949c79845a5d81b09aa1d,171425,0.495721
4,03bf8c511fa949c79845a5d81b09aa1d,252384,0.495721
...,...,...,...
12292583,0d3c58c6864546c689fa0997de484b30,162851,0.984492
12292584,ef559274a81e4ff4b92aa3e9f6805886,136818,0.984492
12292585,c4453128ae584dca8f2a8d20a7c74263,143721,0.984492
12292586,c4453128ae584dca8f2a8d20a7c74263,154423,0.984492


In [10]:
df_g = df_new.copy()
df_g['vacancy_id'] = df_g['vacancy_id_']
df_g['client_id'] = df_g['cookie_id']
df_g

,cookie_id,vacancy_id_,rating,vacancy_id,client_id
0,97990f1a021d4be19aa3f955b7eacab4,129850,0.495721,129850,97990f1a021d4be19aa3f955b7eacab4
1,03bf8c511fa949c79845a5d81b09aa1d,108347,0.495721,108347,03bf8c511fa949c79845a5d81b09aa1d
2,03bf8c511fa949c79845a5d81b09aa1d,109069,0.495721,109069,03bf8c511fa949c79845a5d81b09aa1d
3,03bf8c511fa949c79845a5d81b09aa1d,171425,0.495721,171425,03bf8c511fa949c79845a5d81b09aa1d
4,03bf8c511fa949c79845a5d81b09aa1d,252384,0.495721,252384,03bf8c511fa949c79845a5d81b09aa1d
...,...,...,...,...,...
12292583,0d3c58c6864546c689fa0997de484b30,162851,0.984492,162851,0d3c58c6864546c689fa0997de484b30
12292584,ef559274a81e4ff4b92aa3e9f6805886,136818,0.984492,136818,ef559274a81e4ff4b92aa3e9f6805886
12292585,c4453128ae584dca8f2a8d20a7c74263,143721,0.984492,143721,c4453128ae584dca8f2a8d20a7c74263
12292586,c4453128ae584dca8f2a8d20a7c74263,154423,0.984492,154423,c4453128ae584dca8f2a8d20a7c74263


In [11]:
df_g['cookie_id'] = df_g['cookie_id'].astype("category")
df_g['vacancy_id_'] = df_g['vacancy_id_'].astype("category")
df_g['cookie_id'] = df_g['cookie_id'].cat.codes
df_g['vacancy_id_'] = df_g['vacancy_id_'].cat.codes
df_g

,cookie_id,vacancy_id_,rating,vacancy_id,client_id
0,196572,29849,0.495721,129850,97990f1a021d4be19aa3f955b7eacab4
1,4860,8346,0.495721,108347,03bf8c511fa949c79845a5d81b09aa1d
2,4860,9068,0.495721,109069,03bf8c511fa949c79845a5d81b09aa1d
3,4860,71424,0.495721,171425,03bf8c511fa949c79845a5d81b09aa1d
4,4860,152383,0.495721,252384,03bf8c511fa949c79845a5d81b09aa1d
...,...,...,...,...,...
12292583,17191,62850,0.984492,162851,0d3c58c6864546c689fa0997de484b30
12292584,308843,36817,0.984492,136818,ef559274a81e4ff4b92aa3e9f6805886
12292585,253538,43720,0.984492,143721,c4453128ae584dca8f2a8d20a7c74263
12292586,253538,54422,0.984492,154423,c4453128ae584dca8f2a8d20a7c74263


In [12]:
df_grouped = df_g.copy()
df_grouped = df_grouped.drop(['vacancy_id', 'client_id'], axis=1)
df_grouped = df_grouped.groupby(['cookie_id', 'vacancy_id_']).sum().reset_index()
df_grouped.head(10)

,cookie_id,vacancy_id_,rating
0,0,37658,1.106765
1,0,53974,1.106765
2,0,74952,2.094087
3,0,76170,1.106765
4,0,82444,2.213530
5,0,87528,1.106765
6,1,6675,1.106765
7,1,8689,1.106765
8,1,15743,1.106765
9,1,69614,1.106765


In [13]:
sparse_vacancy_person = sparse.csr_matrix((df_grouped['rating'].astype(float), (df_grouped['vacancy_id_'], df_grouped['cookie_id'])))
sparse_person_vacancy = sparse.csr_matrix((df_grouped['rating'].astype(float), (df_grouped['cookie_id'], df_grouped['vacancy_id_'])))

model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

alpha = 15
data = (sparse_person_vacancy * alpha).astype('double')
model.fit(data)

  0%|          | 0/50 [00:00<?, ?it/s]

In [14]:
content_id = 450
n_similar = 5

person_vecs = model.user_factors
vacancy_vecs = model.item_factors

vacancy_norms = np.sqrt((vacancy_vecs * vacancy_vecs).sum(axis=1))

scores = vacancy_vecs.dot(vacancy_vecs[content_id]) / vacancy_norms
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
similar = sorted(zip(top_idx, scores[top_idx] / vacancy_norms[content_id]), key=lambda x: -x[1])

for content in similar:
    idx, score = content
    print(df_grouped.vacancy_id_.loc[df_grouped.vacancy_id_ == idx].iloc[0])

450
28014
34986
78266
17385


In [15]:
def recommend(person_id, sparse_person_vacancy, person_vecs, vacancy_vecs, num_contents=5):
    person_interactions = sparse_person_vacancy[person_id,:].toarray()
    person_interactions = person_interactions.reshape(-1) + 1
    person_interactions[person_interactions > 1] = 0
    rec_vector = person_vecs[person_id,:].dot(vacancy_vecs.T).toarray()
    
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = person_interactions * rec_vector_scaled
    vacancy_idx = np.argsort(recommend_vector)[::-1][:num_contents]
    
    vacancy = []
    scores = []

    for idx in vacancy_idx:
        vacancy.append(df_grouped.vacancy_id_.loc[df_grouped.vacancy_id_ == idx].iloc[0])
        scores.append(recommend_vector[idx])
    
    return pd.DataFrame({'vacancy_ID': vacancy, 'score': scores})

In [16]:
person_vecs = sparse.csr_matrix(model.user_factors)
vacancy_vecs = sparse.csr_matrix(model.item_factors)

In [17]:
person_id = 0
recommendations = recommend(person_id, sparse_person_vacancy, person_vecs, vacancy_vecs)
print(recommendations)

   vacancy_ID     score
0       82869  1.000000
1       16822  0.987690
2       80381  0.887601
3       69193  0.855261
4       58241  0.845916


In [49]:
def user_id(cookie, dic):
    return dic.cookie_id.loc[dic.client_id == cookie].iloc[0]

In [23]:
df_sample = pd.read_parquet('./data/test_private_sample_submission_mfti.parquet', engine='pyarrow')
df_sample

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]"
1,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]"
2,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]"
3,0099387c921b41e7bae6c99dd8254b60,"[100100, 100101, 100102, 100103, 100104]"
4,009f65e8ae99413a8da94a491320580a,"[100100, 100101, 100102, 100103, 100104]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[100100, 100101, 100102, 100103, 100104]"
3082,ffbc08b528c64f22996873fc63872202,"[100100, 100101, 100102, 100103, 100104]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[100100, 100101, 100102, 100103, 100104]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[100100, 100101, 100102, 100103, 100104]"


In [27]:
print(user_id(df_sample['cookie_id'][1], df_g))

275


In [39]:
df_g.cookie_id.loc[df_g.client_id == 'ffefa79a74804ee69e6c131e0d05b948'].iloc[0]

330110

In [52]:
df_sample_users = []
for elm in df_sample['cookie_id']:
    df_sample_users.append(user_id(elm, df_g))
df_sample_users = pd.DataFrame(df_sample_users)

In [54]:
rec = []
for elm in df_sample_users[0]:
    rec.append(recommend(elm, sparse_person_vacancy, person_vecs, vacancy_vecs))
rec

[   vacancy_ID     score
 0       64601  1.000000
 1       37701  0.990369
 2       82381  0.981363
 3      153816  0.975528
 4      153945  0.971324,
    vacancy_ID     score
 0       38122  1.000000
 1        7525  0.916560
 2       50515  0.910750
 3       35429  0.905675
 4       20187  0.888081,
    vacancy_ID     score
 0      160153  1.000000
 1      153677  0.967441
 2       71331  0.942587
 3      146284  0.905883
 4       16899  0.905793,
    vacancy_ID     score
 0       73336  1.000000
 1       53969  0.900500
 2       76397  0.800122
 3       97406  0.799672
 4       15219  0.797953,
    vacancy_ID     score
 0       38122  1.000000
 1       50515  0.912342
 2        7525  0.853442
 3         430  0.851245
 4       48685  0.844741,
    vacancy_ID     score
 0      102607  1.000000
 1       98113  0.923816
 2       82869  0.907847
 3       11504  0.893841
 4       64601  0.879396,
    vacancy_ID     score
 0       10792  1.000000
 1       10791  0.896729
 2       13304  0.8